# Rasters

Data Source - [Forests](https://globalmaps.github.io/ptc.html)

Data Source - [Surface_Temperature](https://neo.gsfc.nasa.gov/view.php?datasetId=MOD_LSTD_M&year=2021)

In [ ]:
import rasterio
import numpy as np

forests_file = rasterio.open('../resources/gm_ve_v1.tif')
forests = forests_file.read()

print(np.amin(forests))
print(np.amax(forests))
print(len(np.unique(forests)))

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(facecolor='#FCF6F5FF')

ax = plt.axes()
fig.set_size_inches(7, 3.5)
ax.patch.set_facecolor('#FCF6F5FF')
ax.imshow(forests[0],cmap='Greens',interpolation='nearest')
ax.axis('off')
plt.show()

In [ ]:
from matplotlib.colors import ListedColormap, LinearSegmentedColormap, BoundaryNorm
from matplotlib import cm

purples = cm.get_cmap('Greens', 254)
newcolors = purples(np.linspace(0, 1, 254))
background_colour = np.array([0.9882352941176471, 0.9647058823529412, 0.9607843137254902, 1.0])
newcolors[:1, :] = background_colour
newcmp_forests = ListedColormap(newcolors)

bounds = np.arange(254)
norm_forests = BoundaryNorm(bounds, newcmp_forests.N)

gradient = np.linspace(0, 1, 254)
gradient = np.vstack((gradient, gradient))
fig, ax = plt.subplots()
ax.imshow(gradient, aspect='auto', cmap=newcmp_forests)
ax.get_yaxis().set_visible(False)
ax.axvline(100, ls="--", c='black', )
plt.text(105, 0.7, "Data Ends", rotation=90)

plt.show()

In [ ]:
forests[0][forests[0] == 254] = 0.0

ourcmap = cm.get_cmap('Greens', 101)
newcolors = ourcmap(np.linspace(0, 1, 101))
background_colour = np.array([0.9882352941176471, 0.9647058823529412, 0.9607843137254902, 1.0])
newcolors[:1, :] = background_colour
newcmp_forests = ListedColormap(newcolors)

### Exercise 

- Plot the data using the new colourmap and norm

## Projections

In [ ]:
import pyproj 
from pyproj import CRS

for x, y in pyproj.pj_list.items():
    print(x, y)

In [ ]:
import rioxarray as rxr
from rasterio.crs import CRS

org_surface_timp_file = rasterio.open('../resources/MOD_LSTD_M_2021-01-01_rgb_3600x1800.TIFF')
org_surface_temp = org_surface_timp_file.read()

surface_temp_file = rxr.open_rasterio('../resources/MOD_LSTD_M_2021-01-01_rgb_3600x1800.TIFF', masked=True).squeeze()

print("Original projection: ", surface_temp_file.rio.crs)
crs_rob = CRS.from_string('+proj=robin')
surface_temp = surface_temp_file.rio.reproject(crs_rob)
print("New projection: ", surface_temp.rio.crs)
surface_temp = surface_temp.to_numpy()

### Exercise

- Plot the new data and the old data to show the difference in projections
- Fix the colourmap issues
- Generate a new projection

### Exercise

- Go to https://neo.gsfc.nasa.gov/, pick a dataset and generate a new map

![alt text](../resources/assets/movie.gif "Title")